In [1]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.metrics import mean_squared_error,accuracy_score,mean_absolute_error
from sklearn.neural_network import MLPRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import ElasticNet, Lasso,Ridge, LinearRegression,HuberRegressor,SGDRegressor,TweedieRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import LinearSVR,SVR
from sklearn.metrics import mean_squared_error,explained_variance_score,r2_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, AdaBoostRegressor,BaggingRegressor,ExtraTreesRegressor,VotingRegressor
import xgboost as xgb
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_regression

In [2]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM, GRU,Bidirectional,TimeDistributed, ConvLSTM2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.metrics import RootMeanSquaredError,mean_absolute_percentage_error
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [3]:
# split a univariate time series into patterns
def get_Patterns(TSeries, n_inputs,h):
    X,y,z = pd.DataFrame(np.zeros((len(TSeries)-n_inputs-h+1,n_inputs))), pd.DataFrame(), pd.DataFrame()
    for i in range(len(TSeries)):
        # find the end of this pattern
        end_ix = i + n_inputs + h - 1
        # check if we are beyond the time series
        if end_ix > len(TSeries)-1:
            break
        # gather input and output parts of the pattern
        for j in range(n_inputs):
            X.loc[i,j]=TSeries.iloc[i+j,0]
        i=i+n_inputs
        y=y.append(TSeries.iloc[end_ix], ignore_index = True)
        sinX=pd.DataFrame(np.sin(X))
        cosX=pd.DataFrame(np.cos(X))
        squareX=pd.DataFrame(np.power(X,2))
        X1=pd.concat([X,sinX,cosX,squareX], axis=1)
    return np.array(X1),np.array(y)

In [4]:
# originalData should be a Column Vectored DataFrame
def minmaxNorm(originalData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    normalizedData=np.zeros(lenOriginal)   
    normalizedData = []
    for i in range (lenOriginal):
        normalizedData.append((originalData.iloc[i]-min2norm)/(max2norm-min2norm))    
    return pd.DataFrame(normalizedData)

In [5]:
# originalData and forecastedData should be Column Vectored DataFrames
def minmaxDeNorm( originalData, forecastedData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    denormalizedData=[]    
    for i in range (lenOriginal):
        denormalizedData.append((forecastedData.iloc[i]*(max2norm-min2norm))+min2norm)  
    return pd.DataFrame(denormalizedData)

In [6]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findRMSE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainRMSE=0;
    for i in range (lenTrainValidation):
        trainRMSE=trainRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2) 
    trainRMSE=np.sqrt(trainRMSE/lenTrainValidation)

    testRMSE=0;
    for i in range (lenTrainValidation,l,1):
        testRMSE=testRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    testRMSE=np.sqrt(testRMSE/lenTest)
    return trainRMSE, testRMSE 

In [7]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findSMAPE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainSMAPE=0;
    for i in range (lenTrainValidation):
        trainSMAPE=trainSMAPE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/((np.abs(forecasted_value.iloc[i,0])+np.abs(Timeseries_Data.iloc[i,0]))/2))
    
    trainSMAPE=(trainSMAPE/(lenTrainValidation))*100;

    testSMAPE=0;
    for i in range (lenTrainValidation,l,1):
        testSMAPE=testSMAPE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/((np.abs(forecasted_value.iloc[i,0])+np.abs(Timeseries_Data.iloc[i,0]))/2))
    
    testSMAPE=(testSMAPE/lenTest)*100;
    return trainSMAPE, testSMAPE

In [8]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMAE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMAE=0;
    for i in range (lenTrainValidation):
        trainMAE=trainMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]) 
    trainMAE=(trainMAE/(lenTrainValidation));

    testMAE=0;
    for i in range (lenTrainValidation,l,1):
        testMAE=testMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    testMAE=(testMAE/lenTest);
    return trainMAE, testMAE

In [9]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMASE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMASE=0;
    T1=Timeseries_Data.iloc[0:lenTrainValidation,0]
    T2=T1.diff()
    T2=T2.drop(0)
    T2=np.abs(T2)
    M=np.mean(T2)
    for i in range (lenTrainValidation):
        trainMASE=trainMASE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/M) 

    trainMASE=(trainMASE/(lenTrainValidation))

    testMASE=0
    T1=Timeseries_Data.iloc[lenTrainValidation:l,0]
    T2=T1.diff()
    T2=T2.drop(lenTrainValidation)
    T2=np.abs(T2)
    M=np.mean(T2)
    for i in range (lenTrainValidation,l,1):
        testMASE=testMASE+(np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])/M)
    testMASE=(testMASE/lenTest)
    return trainMASE,testMASE 

In [10]:
def get_Activation_Function(no):
    if no==0:
        return "relu"
    elif no==1:
        return "sigmoid"
    elif no==2:
        return "tanh"
    elif no==3:
        return "selu"
    elif no==4:
        return "exponential"
    else:
        return "elu"

In [11]:
def get_kernel_initializer(no):
    if no==0:
        return "glorot_uniform"
    elif no==1:
        return "glorot_normal"
    elif no==2:
        return "he_uniform"
    else:
        return "he_normal"

In [24]:
def Find_Fitness_LSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(LSTM(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [25]:
def Find_Fitness_GRU(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(GRU(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [26]:
def Find_Fitness_CNN(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    LagLength=x.shape[1]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    x = x.reshape((x.shape[0], x.shape[1], n_features))
    model1 = Sequential()
    model1.add(Conv1D(filters=256, kernel_size=2, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [27]:
def Find_Fitness_ConvLSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    n_seq=1
    LagLength=x.shape[1]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    xTrain=xTrain.reshape((xTrain.shape[0], n_seq, 1, LagLength, n_features))
    xValid=xValid.reshape((xValid.shape[0], n_seq, 1, LagLength, n_features))
    xTest=xTest.reshape((xTest.shape[0], n_seq, 1, LagLength, n_features))
    x=x.reshape((x.shape[0], n_seq, 1, LagLength, n_features))
    model1 = Sequential()
    model1.add(ConvLSTM2D(filters=256, kernel_size=(1,2), activation='tanh',kernel_initializer="he_normal" , input_shape=(n_seq,1, LagLength, n_features),name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [28]:
def Find_Fitness_BiLSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    LagLength=x.shape[1]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(Bidirectional(LSTM(256, activation='tanh',kernel_initializer="he_normal"),input_shape=(LagLength,n_features),name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [29]:
def Find_Fitness_CNNLSTM(x,y,lenValid,lenTest,chromosome):
    NOP=len(y)
    n_features=1
    n_seq =1
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    xTrain=xTrain.reshape((xTrain.shape[0], n_seq, LagLength, n_features))
    xValid=xValid.reshape((xValid.shape[0], n_seq, LagLength, n_features))
    xTest=xTest.reshape((xTest.shape[0], n_seq, LagLength, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=256, kernel_size=1, activation='tanh',kernel_initializer="he_normal"),input_shape=(None, LagLength, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(16, activation='relu',kernel_initializer="he_normal"))
    model.add(Flatten())
    model.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model.compile(loss=MeanSquaredError(),optimizer=Adam(),metrics=[RootMeanSquaredError(),mean_absolute_percentage_error])
    # fit
    model.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, batch_size=16, verbose=1,callbacks=[cp])
    yhatNorm=model.predict(xTest).flatten().reshape(xTest.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [30]:
def Find_Fitness_Stacked_LSTM(x,y,lenValid,lenTest,chromosome):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model = Sequential()
    model.add(LSTM(256, activation='tanh',kernel_initializer="he_normal", input_shape=(LagLength,n_features),return_sequences=True))
    model.add(LSTM(16, activation='relu', kernel_initializer="he_normal"))
    model.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model.compile(loss=MeanSquaredError(),optimizer=Adam(),metrics=[RootMeanSquaredError(),mean_absolute_percentage_error])
    # fit
    model.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=10, batch_size=16, verbose=1,callbacks=[cp])
    yhatNorm=model.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [19]:
def Find_Fitness(x,y,lenValid,lenTest,model):
    NOP=y.shape[0]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:(lenTrain+lenValid),:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:(lenTrain+lenValid),0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model.fit(xTrain, yTrain)
    yhatNorm=model.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [20]:
def findUOD(TimeSeries):
    Dmin=np.min(TimeSeries)
    Dmax=np.max(TimeSeries)
    D1=0.2*(Dmax-Dmin)
    D2=0.2*(Dmax-Dmin)
    U=pd.DataFrame(np.zeros((1,2)))
    U.iloc[0,0]=Dmin-D1
    U.iloc[0,1]=Dmax+D2
    return pd.DataFrame(U)

In [21]:
#Read the Time Series Dataset
Timeseries_Data=pd.read_csv('AQI.csv',header=None)
LagLength=24
h=1
NumberOfRepeatitions=20
lt=Timeseries_Data.shape[0]
lenTrain=int(round(lt*0.6))
lenValidation=int(round(lt*0.2))
lenTest=int(lt-lenTrain-lenValidation)
# NORMALIZE THE DATA
normalizedData=minmaxNorm(Timeseries_Data,lenTrain+lenValidation);
# Transform the Time Series into Patterns Using Sliding Window
X, y = get_Patterns(normalizedData, LagLength, h)
MeanAccuracyT=pd.DataFrame()
StdAccuracyT=pd.DataFrame()
for j in range(0,28,1):
    print('Model:::::::::::::::::::::::::::::::',j)
    if j==0:         
        model=LinearRegression()
        name='1.LinearRegression'
    elif j==1:
        model=Lasso()
        name='2.Lasso'
    elif j==2:
        model=Ridge()
        name='3.Ridge'
    elif j==3:
        model=ElasticNet()
        name='4.ElasticNet'
    elif j==4:
        model=HuberRegressor()
        name='5.HuberRegressor'
    elif j==5:
        model=SGDRegressor()
        name='6.SGDRegressor'
    elif j==6:
        model=TweedieRegressor()
        name='7.TweedieRegressor'
    elif j==7:
        model=AdaBoostRegressor()
        name='8.AdaBoostRegressor'
    elif j==8:
        model=RandomForestRegressor()
        name='9.RandomForestRegressor'
    elif j==9:
        model=GradientBoostingRegressor()
        name='10.GradientBoostingRegressor'
    elif j==10:
        model=LinearSVR()
        name='11.LinearSVR'
    elif j==11:
        model=MLPRegressor()
        name='12.MLP'
    elif j==12:
        model=SVR()
        name='13.SVR'
    elif j==13:
        model=ExtraTreesRegressor()
        name='14.ExtraTreesRegressor'
    elif j==14:
        model=BaggingRegressor()
        name='15.BaggingRegressor'
    elif j==15:
        model=DecisionTreeRegressor()
        name='16.DecisionTreeRegressor'
    elif j==16:
        model=KNeighborsRegressor()
        name='17.KNeighborsRegressor'
    elif j==17:
        model=xgb.XGBRegressor(silent=True)
        name='18.XGB'
    elif j==18:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=xgb.XGBRegressor(silent=True))
        name='19.Lasso_Tweedi_xgb'
    elif j==19:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=LinearSVR())
        name='20.Lasso_Tweedi_LinearSVR'
    elif j==20:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=Lasso())
        name='21.Lasso_Tweedi_Lasso'
    elif j==21:
        name='22.LSTM'
    elif j==22:
        name='23.CNN'
    elif j==23:
        name='24.BiLSTM' 
    elif j==24:
        name='25.GRU'
    elif j==25:
        name='26.ConvLSTM'
    elif j==26:
        name='27.CNN_LSTM'
    elif j==27:
        name='28.Stacked_LSTM'
    else:
        print('Completed.....................')

    file1='./'+str(name)+"_Accuracy.xlsx"
    file2='./'+str(name)+"_Forecasts.xlsx"
    Forecasts=pd.DataFrame()
    Accuracy=pd.DataFrame()
    for i in range(NumberOfRepeatitions):
        print('No. of repeatitions completed:',i)
        if j<=20:
            ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
        elif j==21:
            ynorm1=Find_Fitness_LSTM(X,y,lenValidation,lenTest)
        elif j==22:
            ynorm1=Find_Fitness_CNN(X,y,lenValidation,lenTest)
        elif j==23:
            ynorm1=Find_Fitness_BiLSTM(X,y,lenValidation,lenTest) 
        elif j==24:
            ynorm1=Find_Fitness_GRU(X,y,lenValidation,lenTest)
        elif j==25:
            ynorm1=Find_Fitness_ConvLSTM(X,y,lenValidation,lenTest)
        elif j==26:
            ynorm1=Find_Fitness_CNNLSTM(X,y,lenValidation,lenTest)
        elif j==27:
            ynorm1=Find_Fitness_Stacked_LSTM(X,y,lenValidation,lenTest)
        else:
            print('Completed.....................')
        ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
        ynorm=ynorm.append(ynorm1,ignore_index = True)
        yhat=minmaxDeNorm(Timeseries_Data, ynorm, lenTrain+lenValidation)
        Accuracy.loc[i,0],Accuracy.loc[i,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,2],Accuracy.loc[i,3]=findSMAPE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,4],Accuracy.loc[i,5]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,6],Accuracy.loc[i,7]=findMASE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Forecasts=Forecasts.append(yhat.T,ignore_index = True)
    Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
    Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
    MeanAccuracy=pd.DataFrame(np.mean(Accuracy))    
    MeanAccuracyT=MeanAccuracyT.append(MeanAccuracy.T, ignore_index = True)
    StdAccuracy=pd.DataFrame(np.std(Accuracy))    
    StdAccuracyT=StdAccuracyT.append(StdAccuracy.T, ignore_index = True)
    del Accuracy
    del Forecasts
    del MeanAccuracy
    del StdAccuracy
MeanAccuracyT.to_excel('All_Model_Mean_Accuracy.xlsx',sheet_name='All_Model_Accuracy',index=False)
del MeanAccuracyT
StdAccuracyT.to_excel('All_Model_Stdev_Accuracy.xlsx',sheet_name='All_Model_Stdev_Accuracy',index=False)
del StdAccuracyT

NameError: name 'Find_Features_CNN' is not defined

In [31]:
NumberOfRepeatitions=10
MeanAccuracyT=pd.DataFrame()
StdAccuracyT=pd.DataFrame()
for j in range(22,28,1):
    print('Model:::::::::::::::::::::::::::::::',j)
    if j==0:         
        model=LinearRegression()
        name='1.LinearRegression'
    elif j==1:
        model=Lasso()
        name='2.Lasso'
    elif j==2:
        model=Ridge()
        name='3.Ridge'
    elif j==3:
        model=ElasticNet()
        name='4.ElasticNet'
    elif j==4:
        model=HuberRegressor()
        name='5.HuberRegressor'
    elif j==5:
        model=SGDRegressor()
        name='6.SGDRegressor'
    elif j==6:
        model=TweedieRegressor()
        name='7.TweedieRegressor'
    elif j==7:
        model=AdaBoostRegressor()
        name='8.AdaBoostRegressor'
    elif j==8:
        model=RandomForestRegressor()
        name='9.RandomForestRegressor'
    elif j==9:
        model=GradientBoostingRegressor()
        name='10.GradientBoostingRegressor'
    elif j==10:
        model=LinearSVR()
        name='11.LinearSVR'
    elif j==11:
        model=MLPRegressor()
        name='12.MLP'
    elif j==12:
        model=SVR()
        name='13.SVR'
    elif j==13:
        model=ExtraTreesRegressor()
        name='14.ExtraTreesRegressor'
    elif j==14:
        model=BaggingRegressor()
        name='15.BaggingRegressor'
    elif j==15:
        model=DecisionTreeRegressor()
        name='16.DecisionTreeRegressor'
    elif j==16:
        model=KNeighborsRegressor()
        name='17.KNeighborsRegressor'
    elif j==17:
        model=xgb.XGBRegressor(silent=True)
        name='18.XGB'
    elif j==18:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=xgb.XGBRegressor(silent=True))
        name='19.Lasso_Tweedi_xgb'
    elif j==19:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=LinearSVR())
        name='20.Lasso_Tweedi_LinearSVR'
    elif j==20:
        model=StackingRegressor(regressors=[Lasso(),TweedieRegressor()],meta_regressor=Lasso())
        name='21.Lasso_Tweedi_Lasso'
    elif j==21:
        name='22.LSTM'
    elif j==22:
        name='23.CNN'
    elif j==23:
        name='24.BiLSTM' 
    elif j==24:
        name='25.GRU'
    elif j==25:
        name='26.ConvLSTM'
    elif j==26:
        name='27.CNN_LSTM'
    elif j==27:
        name='28.Stacked_LSTM'
    else:
        print('Completed.....................')

    file1='./'+str(name)+"_Accuracy.xlsx"
    file2='./'+str(name)+"_Forecasts.xlsx"
    Forecasts=pd.DataFrame()
    Accuracy=pd.DataFrame()
    for i in range(NumberOfRepeatitions):
        print('No. of repeatitions completed:',i)
        if j<=20:
            ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
        elif j==21:
            ynorm1=Find_Fitness_LSTM(X,y,lenValidation,lenTest)
        elif j==22:
            ynorm1=Find_Fitness_CNN(X,y,lenValidation,lenTest)
        elif j==23:
            ynorm1=Find_Fitness_BiLSTM(X,y,lenValidation,lenTest) 
        elif j==24:
            ynorm1=Find_Fitness_GRU(X,y,lenValidation,lenTest)
        elif j==25:
            ynorm1=Find_Fitness_ConvLSTM(X,y,lenValidation,lenTest)
        elif j==26:
            ynorm1=Find_Fitness_CNNLSTM(X,y,lenValidation,lenTest)
        elif j==27:
            ynorm1=Find_Fitness_Stacked_LSTM(X,y,lenValidation,lenTest)
        else:
            print('Completed.....................')
        ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
        ynorm=ynorm.append(ynorm1,ignore_index = True)
        yhat=minmaxDeNorm(Timeseries_Data, ynorm, lenTrain+lenValidation)
        Accuracy.loc[i,0],Accuracy.loc[i,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,2],Accuracy.loc[i,3]=findSMAPE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,4],Accuracy.loc[i,5]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Accuracy.loc[i,6],Accuracy.loc[i,7]=findMASE( Timeseries_Data,yhat,lenTrain+lenValidation)
        Forecasts=Forecasts.append(yhat.T,ignore_index = True)
    Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
    Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
    MeanAccuracy=pd.DataFrame(np.mean(Accuracy))    
    MeanAccuracyT=MeanAccuracyT.append(MeanAccuracy.T, ignore_index = True)
    StdAccuracy=pd.DataFrame(np.std(Accuracy))    
    StdAccuracyT=StdAccuracyT.append(StdAccuracy.T, ignore_index = True)
    del Accuracy
    del Forecasts
    del MeanAccuracy
    del StdAccuracy
MeanAccuracyT.to_excel('All_Model_Mean_Accuracy.xlsx',sheet_name='All_Model_Accuracy',index=False)
del MeanAccuracyT
StdAccuracyT.to_excel('All_Model_Stdev_Accuracy.xlsx',sheet_name='All_Model_Stdev_Accuracy',index=False)
del StdAccuracyT

Model::::::::::::::::::::::::::::::: 22
No. of repeatitions completed: 0
Epoch 1/10
1806/1806 [==============================] - 8s 4ms/step - loss: 5.5430 - root_mean_squared_error: 2.3544 - val_loss: 0.0487 - val_root_mean_squared_error: 0.2206
Epoch 2/10
1806/1806 [==============================] - 7s 4ms/step - loss: 0.0254 - root_mean_squared_error: 0.1592 - val_loss: 0.0285 - val_root_mean_squared_error: 0.1690
Epoch 3/10
1806/1806 [==============================] - 7s 4ms/step - loss: 0.0249 - root_mean_squared_error: 0.1579 - val_loss: 0.0278 - val_root_mean_squared_error: 0.1669
Epoch 4/10
1806/1806 [==============================] - 7s 4ms/step - loss: 0.0250 - root_mean_squared_error: 0.1580 - val_loss: 0.0278 - val_root_mean_squared_error: 0.1668
Epoch 5/10
1806/1806 [==============================] - 7s 4ms/step - loss: 0.0250 - root_mean_squared_error: 0.1582 - val_loss: 0.0282 - val_root_mean_squared_error: 0.1678
Epoch 6/10
1806/1806 [==============================] - 7

1806/1806 [==============================] - 9s 5ms/step - loss: 0.0250 - root_mean_squared_error: 0.1581 - val_loss: 0.0279 - val_root_mean_squared_error: 0.1670
Epoch 6/10
1806/1806 [==============================] - 9s 5ms/step - loss: 0.0251 - root_mean_squared_error: 0.1583 - val_loss: 0.0277 - val_root_mean_squared_error: 0.1665
Epoch 7/10
1806/1806 [==============================] - 9s 5ms/step - loss: 0.0251 - root_mean_squared_error: 0.1585 - val_loss: 0.0280 - val_root_mean_squared_error: 0.1673
Epoch 8/10
1806/1806 [==============================] - 10s 5ms/step - loss: 0.0251 - root_mean_squared_error: 0.1584 - val_loss: 0.0302 - val_root_mean_squared_error: 0.1737
Epoch 9/10
1806/1806 [==============================] - 12s 7ms/step - loss: 0.0251 - root_mean_squared_error: 0.1585 - val_loss: 0.0282 - val_root_mean_squared_error: 0.1679
Epoch 10/10
1506/1506 [==============================] - 5s 3ms/step
No. of repeatitions completed: 5
Epoch 1/10
1806/1806 [===============

Epoch 10/10
1506/1506 [==============================] - 5s 3ms/step
No. of repeatitions completed: 9
Epoch 1/10
1806/1806 [==============================] - 9s 5ms/step - loss: 12.8144 - root_mean_squared_error: 3.5797 - val_loss: 0.0279 - val_root_mean_squared_error: 0.1669
Epoch 2/10
1806/1806 [==============================] - 11s 6ms/step - loss: 0.0249 - root_mean_squared_error: 0.1579 - val_loss: 0.0280 - val_root_mean_squared_error: 0.1675
Epoch 3/10
1806/1806 [==============================] - 10s 6ms/step - loss: 0.0249 - root_mean_squared_error: 0.1579 - val_loss: 0.0284 - val_root_mean_squared_error: 0.1684
Epoch 4/10
1806/1806 [==============================] - 11s 6ms/step - loss: 0.0250 - root_mean_squared_error: 0.1580 - val_loss: 0.0279 - val_root_mean_squared_error: 0.1672
Epoch 5/10
1806/1806 [==============================] - 10s 6ms/step - loss: 0.0250 - root_mean_squared_error: 0.1581 - val_loss: 0.0285 - val_root_mean_squared_error: 0.1688
Epoch 6/10
1806/1806 [=

Epoch 3/10
1806/1806 [==============================] - 371s 205ms/step - loss: 0.0015 - root_mean_squared_error: 0.0384 - val_loss: 2.5147e-04 - val_root_mean_squared_error: 0.0159
Epoch 4/10
1806/1806 [==============================] - 393s 217ms/step - loss: 6.1269e-04 - root_mean_squared_error: 0.0248 - val_loss: 5.1190e-04 - val_root_mean_squared_error: 0.0226
Epoch 5/10
1806/1806 [==============================] - 345s 191ms/step - loss: 4.3468e-04 - root_mean_squared_error: 0.0208 - val_loss: 6.3768e-05 - val_root_mean_squared_error: 0.0080
Epoch 6/10
1806/1806 [==============================] - 290s 161ms/step - loss: 4.0498e-04 - root_mean_squared_error: 0.0201 - val_loss: 7.9453e-05 - val_root_mean_squared_error: 0.0089
Epoch 7/10
1806/1806 [==============================] - 358s 198ms/step - loss: 3.5926e-04 - root_mean_squared_error: 0.0190 - val_loss: 7.3048e-05 - val_root_mean_squared_error: 0.0085
Epoch 8/10
1806/1806 [==============================] - 373s 207ms/step - 

1806/1806 [==============================] - 387s 214ms/step - loss: 0.0028 - root_mean_squared_error: 0.0528 - val_loss: 8.3927e-05 - val_root_mean_squared_error: 0.0092
Epoch 6/10
1806/1806 [==============================] - 377s 209ms/step - loss: 3.9799e-04 - root_mean_squared_error: 0.0199 - val_loss: 4.0999e-05 - val_root_mean_squared_error: 0.0064
Epoch 7/10
1806/1806 [==============================] - 368s 204ms/step - loss: 4.3208e-04 - root_mean_squared_error: 0.0208 - val_loss: 2.3282e-04 - val_root_mean_squared_error: 0.0153
Epoch 8/10
1806/1806 [==============================] - 379s 210ms/step - loss: 4.3058e-04 - root_mean_squared_error: 0.0208 - val_loss: 1.1468e-04 - val_root_mean_squared_error: 0.0107
Epoch 9/10
1806/1806 [==============================] - 313s 173ms/step - loss: 4.0195e-04 - root_mean_squared_error: 0.0200 - val_loss: 1.3720e-04 - val_root_mean_squared_error: 0.0117
Epoch 10/10
1506/1506 [==============================] - 158s 103ms/step
No. of repea

...

1806/1806 [==============================] - 219s 121ms/step - loss: 5.2306e-04 - root_mean_squared_error: 0.0229 - val_loss: 1.1767e-04 - val_root_mean_squared_error: 0.0108
Epoch 10/10
1506/1506 [==============================] - 66s 43ms/step
No. of repeatitions completed: 6
Epoch 1/10
1806/1806 [==============================] - 223s 120ms/step - loss: 0.0017 - root_mean_squared_error: 0.0413 - val_loss: 8.6173e-05 - val_root_mean_squared_error: 0.0093
Epoch 2/10
1806/1806 [==============================] - 192s 106ms/step - loss: 3.5617e-04 - root_mean_squared_error: 0.0189 - val_loss: 9.5848e-05 - val_root_mean_squared_error: 0.0098
Epoch 3/10
1806/1806 [==============================] - 212s 117ms/step - loss: 3.5908e-04 - root_mean_squared_error: 0.0189 - val_loss: 2.3817e-04 - val_root_mean_squared_error: 0.0154
Epoch 4/10
1806/1806 [==============================] - 202s 112ms/step - loss: 3.8931e-04 - root_mean_squared_error: 0.0197 - val_loss: 2.4435e-04 - val_root_mean_squ

Model::::::::::::::::::::::::::::::: 25
No. of repeatitions completed: 0
Epoch 1/10
1806/1806 [==============================] - 99s 51ms/step - loss: 0.8330 - root_mean_squared_error: 0.9127 - val_loss: 0.0288 - val_root_mean_squared_error: 0.1696
Epoch 2/10
1806/1806 [==============================] - 112s 62ms/step - loss: 0.0249 - root_mean_squared_error: 0.1579 - val_loss: 0.0280 - val_root_mean_squared_error: 0.1672
Epoch 3/10
1806/1806 [==============================] - 116s 64ms/step - loss: 0.0250 - root_mean_squared_error: 0.1580 - val_loss: 0.0319 - val_root_mean_squared_error: 0.1786
Epoch 4/10
1806/1806 [==============================] - 116s 64ms/step - loss: 0.0250 - root_mean_squared_error: 0.1582 - val_loss: 0.0289 - val_root_mean_squared_error: 0.1699
Epoch 5/10
1806/1806 [==============================] - 113s 62ms/step - loss: 0.0251 - root_mean_squared_error: 0.1584 - val_loss: 0.0279 - val_root_mean_squared_error: 0.1672
Epoch 6/10
1806/1806 [=====================

Epoch 5/10
1806/1806 [==============================] - 137s 76ms/step - loss: 0.0251 - root_mean_squared_error: 0.1584 - val_loss: 0.0292 - val_root_mean_squared_error: 0.1707
Epoch 6/10
1806/1806 [==============================] - 129s 71ms/step - loss: 0.0251 - root_mean_squared_error: 0.1585 - val_loss: 0.0283 - val_root_mean_squared_error: 0.1683
Epoch 7/10
1806/1806 [==============================] - 137s 76ms/step - loss: 0.0251 - root_mean_squared_error: 0.1585 - val_loss: 0.0299 - val_root_mean_squared_error: 0.1729
Epoch 8/10
1806/1806 [==============================] - 162s 89ms/step - loss: 0.0251 - root_mean_squared_error: 0.1584 - val_loss: 0.0304 - val_root_mean_squared_error: 0.1745
Epoch 9/10
1806/1806 [==============================] - 163s 90ms/step - loss: 0.0251 - root_mean_squared_error: 0.1584 - val_loss: 0.0284 - val_root_mean_squared_error: 0.1685
Epoch 10/10
1506/1506 [==============================] - 52s 34ms/step
No. of repeatitions completed: 5
Epoch 1/10


1806/1806 [==============================] - 143s 79ms/step - loss: 0.0251 - root_mean_squared_error: 0.1583 - val_loss: 0.0263 - val_root_mean_squared_error: 0.1621
Epoch 10/10
1506/1506 [==============================] - 53s 34ms/step
No. of repeatitions completed: 9
Epoch 1/10
1806/1806 [==============================] - 174s 90ms/step - loss: 0.4723 - root_mean_squared_error: 0.6873 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0409
Epoch 2/10
1806/1806 [==============================] - 161s 89ms/step - loss: 7.4068e-04 - root_mean_squared_error: 0.0272 - val_loss: 9.5734e-05 - val_root_mean_squared_error: 0.0098
Epoch 3/10
1806/1806 [==============================] - 138s 76ms/step - loss: 8.4126e-04 - root_mean_squared_error: 0.0290 - val_loss: 0.0021 - val_root_mean_squared_error: 0.0457
Epoch 4/10
1806/1806 [==============================] - 161s 89ms/step - loss: 7.9983e-04 - root_mean_squared_error: 0.0283 - val_loss: 5.4076e-04 - val_root_mean_squared_error: 0.0233
Ep

TypeError: Find_Fitness_CNNLSTM() missing 1 required positional argument: 'chromosome'